# Library load

In [ ]:
import sys, os
import requests #(네이버) api 이용할 때 쓰는 패키지
import selenium #크롤링 패키지

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup #크롤링 패키지
import re #정규식 표현 모듈
from datetime import datetime
import pickle, json, glob, time #pickle: 객체를 파일로 저장해주는 모듈, json: 네이버 api, glob: 파일들 리스트 뽑을 때 사용
# https://wikidocs.net/83
from tqdm import tqdm #진행표시바


import urllib.request #파싱을 위한 모듈

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np

## ii) csv values를 위한 조회
cat이 cd로 출력되는데 이것은 나중에 `catinfo.csv`의 열들과 mapping을 하여 이름으로 바꿔주기로 한다.

In [ ]:
CIDs = pd.read_csv('cids.csv')
cids = list(CIDs['cid'].values)
len(cids)

### 함수로 api 호출하기

In [ ]:
# 각 오퍼레이션 별로 워드에서 검색했을 때, table의 항목구분 열에 1이라고 표시돼있는 필수 파라미터를 다 작성하지 않을 경우 soup 출력시 에러남.

API_HOST = 'http://apis.data.go.kr/B551011/KorService/'
key = '키키키키키키키키키키키키키키키키' # Encoding key

## 오퍼레이션 담은 dic
op = {}
op['det'] = 'detailCommon' # 공통정보조회

# 선택 조회
fimg = 'firstImageYN' # 대표이미지조회
ov = 'overviewYN' # 개요조회
catcd = 'catcodeYN' # 서비스분류코드조회
ar = 'areacodeYN' # 지역코드조회
addr = 'addrinfoYN'# 주소조회
mapin4 = 'mapinfoYN'# 좌표조회


def api(cid_):
    # p81, content id가 ~~인 공통정보의 '기본정보' '대표이미지' '개요' 조회
    url = f'{API_HOST}{op["det"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&contentId={cid_}&defaultYN=Y&{fimg}=Y&{ov}=Y&{catcd}=Y&{mapin4}=Y'#&{ar}=Y&{addr}=Y
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'xml') # parser는 'lxml'로도 가능.

    ## values
    if soup.firstimage.string:
        imgurl1 = soup.firstimage.string
    else:
        imgurl1 = np.NaN

    if soup.firstimage2.string:
        imgurl2 = soup.firstimage2.string
    else:
        imgurl2 = np.NaN
    ovvw = soup.overview.string
    cat1_ = soup.cat1.string
    cat2_ = soup.cat2.string
    cat3_ = soup.cat3.string
    
    ## tmp_df
    tmp_df_ = pd.DataFrame({'cid' : [cid_],
                            'imgurl1' : [imgurl1],
                            'imgurl2' : [imgurl2],
                            'overview' : [ovvw],
                            'cat1' : [cat1_],
                            'cat2' : [cat2_],
                            'cat3' : [cat3_]})
    
    return tmp_df_

### 자동화

In [ ]:
cnt = 0
df = pd.DataFrame()
for enu, CID in tqdm(enumerate(cids[17004:])):
    try:
        tmp_df = api(CID)
    except:
        print(cids.index(CID)) # 호출하지 못한 CID 출력해주세요
        continue # 다음 iter로 넘어가기
        # except문에 동일한 요청이 아닌 다음 loop가 돌아야함.
        # 그러려면 continue를 해줘야 하는데 그럼 try부터 도는데, 그게 맞나..?
        # 말하다보니까 맞다는걸 깨달음.
    
    ## concat & df update
    df = pd.concat([df, tmp_df], ignore_index = True)

In [ ]:
df.shape
# df.head()
df.tail()

In [ ]:
# 맨 처음에 df 생성시 실행하는 코드, 기존 example_df.csv와 병합하기 전에 실행하지 않도록 주의하십쇼
##### df.to_csv('example_df.csv', index=False)

In [ ]:
ex_df = pd.read_csv('example_df.csv')
ex_df.shape

In [ ]:
# ex_df.head()
# ex_df['imgurl2'].isna().sum()

#### 병합 & 저장

In [ ]:
df = pd.concat([ex_df, df], axis = 0, ignore_index = True)

In [ ]:
df.to_csv('example_df.csv', index=False)

### 아래 cell은 일일이 호출하던 original 코드임.
- 이거 대신 함수로 api 호출할 것임.
- 아래 cell은 다 호출하고 호출하지 못한 cid들에 대한 값들 받을 때 사용할 지도 모르니 남겨뒀습니다.
- 사실 필요없는 쓰레긴데 남겨둔 것 입니다.

In [ ]:
# 각 오퍼레이션 별로 워드에서 검색했을 때, table의 항목구분 열에 1이라고 표시돼있는 필수 파라미터를 다 작성하지 않을 경우 soup 출력시 에러남.

API_HOST = 'http://apis.data.go.kr/B551011/KorService/'
key = '키키키키키키키키키키키키키키키키' # Encoding key

## 오퍼레이션 담은 dic
op = {}
op['det'] = 'detailCommon' # 공통정보조회
op['cat'] = 'categoryCode' # catcd name 조회


# 선택 조회
fimg = 'firstImageYN' # 대표이미지조회
ov = 'overviewYN' # 개요조회
catcd = 'catcodeYN' # 서비스분류코드조회
ar = 'areacodeYN' # 지역코드조회
addr = 'addrinfoYN'# 주소조회
mapin4 = 'mapinfoYN'# 좌표조회



# CID = cids[0]
# # csv values를 위한 조회
# url = f'{API_HOST}{op["det"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&contentId={CID}&defaultYN=Y&{fimg}=Y&{ov}=Y&{catcd}=Y&{ar}=Y&{addr}=Y&{mapin4}=Y'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'xml') # parser는 'lxml'로도 가능.


cnt = 0
df = pd.DataFrame()
for enu, CID in tqdm(enumerate(cids[17004:])):
    # p81, content id가 ~~인 공통정보의 '기본정보' '대표이미지' '개요' 조회
    url = f'{API_HOST}{op["det"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&contentId={CID}&defaultYN=Y&{fimg}=Y&{ov}=Y&{catcd}=Y&{mapin4}=Y'#&{ar}=Y&{addr}=Y
    response = requests.get(url)
#     if (response.status_code == 200) | (response.ok) :
#         print('Request OK')
#     else:
#         print(response.status_code)
    
    soup = BeautifulSoup(response.text, 'xml') # parser는 'lxml'로도 가능.
    
#     soup
    
    ## values
    if soup.firstimage.string:
        imgurl1 = soup.firstimage.string
    else:
        imgurl1 = np.NaN

    if soup.firstimage2.string:
        imgurl2 = soup.firstimage2.string
    else:
        imgurl2 = np.NaN
    ovvw = soup.overview.string
    cat1_ = soup.cat1.string
    cat2_ = soup.cat2.string
    cat3_ = soup.cat3.string
    
    ## tmp_df
    tmp_df = pd.DataFrame({'cid' : [CID],
                           'imgurl1' : [imgurl1],
                           'imgurl2' : [imgurl2],
                           'overview' : [ovvw],
                           'cat1' : [cat1_],
                           'cat2' : [cat2_],
                           'cat3' : [cat3_]})
    ## concat & df update
    df = pd.concat([df, tmp_df], ignore_index = True)
#     time.sleep(0.8)
    
#     cnt
#     cnt+=1

#     if enu > 2: # break를 거는 위치가 코드가 다 끝난 다음이니까 2번째까지는 하고 멈추라는 거임.
#         break